In [8]:
import sys
import os

# Get the absolute path to the project directory
project_dir = os.path.abspath("..")

# Append the project directory to sys.path
if project_dir not in sys.path:
    sys.path.append(project_dir)

In [9]:
import sys
import os

# Get the absolute path to the project directory
project_dir = os.path.abspath("..")

# Append the project directory to sys.path
if project_dir not in sys.path:
    sys.path.append(project_dir)

import yfinance as yf
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import warnings
from dataclasses import asdict, is_dataclass, dataclass
import os
import json
import pickle
from pandas.api.types import is_datetime64_any_dtype
import numpy as np
from typing import Dict
import polars as pl

from src.common.AssetData import AssetData
from src.common.AssetDataPolars import AssetDataPolars
from src.common.AssetDataService import AssetDataService
from src.common.AssetFileInOut import AssetFileInOut 
from src.featureAlchemy.FeatureFinancialData import FeatureFinancialData
from src.featureAlchemy.FeatureFinancialDataTS import FeatureFinancialDataTS


In [10]:
assets=AssetFileInOut("../src/stockGroups/bin").loadDictFromFile("group_debug")

# Convert to Polars for speedup
assetspl: Dict[str, AssetDataPolars] = {}
for ticker, asset in assets.items():
    assetspl[ticker] = AssetDataService.to_polars(asset)
    
ticker = "AAPL"
asset = assetspl[ticker]

fin_quar = asset.financials_quarterly
fin_ann = asset.financials_annually
asset_about = asset.about

In [11]:
print(fin_quar.columns)
params = {
    "timesteps": 50,
    'lagList': [1,2,3,5,9,21,63,121,210],
}
ffd = FeatureFinancialData(asset, params=params)
print(asset.shareprice.columns)
asShare = ffd.shareprice
finquar = asset.financials_quarterly
finQuarJoin = ffd.fin_quar_join

['fiscalDateEnding', 'reportedDate', 'reportedEPS', 'estimatedEPS', 'surprise', 'surprisePercentage', 'reportTime', 'grossProfit', 'totalRevenue', 'ebit', 'ebitda', 'totalAssets', 'totalCurrentLiabilities', 'totalShareholderEquity', 'commonStockSharesOutstanding', 'operatingCashflow']
['Date', 'Open', 'High', 'Low', 'Close', 'AdjClose', 'Volume', 'Dividends', 'Splits']


In [12]:
workdays = pd.date_range(start=dt.date(2020, 1, 1), end=dt.date(2020, 12, 31), freq='B')
dates = [date.date() for date in workdays]
featureNames = ffd.getFeatureNames()
features = ffd.apply(dates)

In [13]:
rev = asset.financials_quarterly['totalRevenue'].last()
ebit = asset.financials_quarterly['ebit'].last()
ebitda = asset.financials_quarterly['ebitda'].last()

ebit_nivrev = ebit / rev
ebitda_nivrev = ebitda / rev

print(f"EBIT/Revenue: {ebit_nivrev}\n"
      f"EBITDA/Revenue: {ebitda_nivrev}")

EBIT/Revenue: 0.31029058610094484
EBITDA/Revenue: 0.33819566060885703


In [14]:
ffdts = FeatureFinancialDataTS(asset, params=params)
featureNamesTS = ffdts.getFeatureNames()
featuresTS = ffdts.apply(dates)
featureTS_zero = featuresTS[0]